# TAREA II - SEP I/A
<div style="text-align: justify;">

**PARTICIPANTES DEL PROYECTO:** NICOLÁS GONZÁLEZ - IVÁN TAPIA

In [32]:
import numpy as np

def calcular_matriz_admitancia(num_barras, datos_barras):
    # Inicializar matriz de admitancia Y_bus
    Y_bus = np.zeros((num_barras, num_barras), dtype=complex)
    # Procesar cada barra para llenar la matriz Y_bus
    for i in range(num_barras):
        tipo_barra = datos_barras[i]['tipo']
        if tipo_barra == 'slack':
            # Barra slack (barra de referencia)
            Y_bus[i, i] = 1e9  # Valor alto para mantener fijo el voltaje
        else:
            # Calcular admitancia shunt (si es barra PQ o PV)
            Y_shunt = 1 / (datos_barras[i]['R'] + 1j * datos_barras[i]['X'])
            # Llenar diagonal principal de Y_bus con admitancias shunt
            Y_bus[i, i] = Y_shunt
            # Llenar elementos correspondientes a las conexiones entre barras
            for j in range(num_barras):
                if i != j:
                    if tipo_barra == 'PQ' and datos_barras[j]['tipo'] == 'PQ':
                        # Si ambas son PQ
                        Y_bus[i, j] = -1 / (datos_barras[i]['R'] + 1j * datos_barras[i]['X'])
                    elif (tipo_barra == 'PQ' and datos_barras[j]['tipo'] == 'PV') or (tipo_barra == 'PV' and datos_barras[j]['tipo'] == 'PQ'):
                        # Si es PQ-PV o PV-PQ
                        Y_bus[i, j] = 1 / (datos_barras[i]['R'] + 1j * datos_barras[i]['X'])
    print('Matriz Y_bus antes de resolver:')
    print(Y_bus)
    return Y_bus

def calcular_potencia_inyectada(num_barras, datos_barras):
    P_iny = np.zeros(num_barras)
    Q_iny = np.zeros(num_barras)
    
    for i in range(num_barras):
        P_iny[i] = datos_barras[i]['P']
        Q_iny[i] = datos_barras[i]['Q']
    
    return P_iny, Q_iny

def calcular_jacobiano(num_barras, Y_bus, V, theta):
    J = np.zeros((2*num_barras, 2*num_barras))
    
    for i in range(num_barras):
        for j in range(num_barras):
            if i == j and i != 0:  # Excluir la barra slack (barra 0)
                J[i, j] = -Y_bus[i, i].real * V[i]**2 - sum(Y_bus[i, k] * V[i] * V[k] * np.sin(theta[i] - theta[k]) for k in range(num_barras if j != i else 0))
                J[i, j + num_barras] = -sum(Y_bus[i, k] * V[i] * V[k] * np.cos(theta[i] - theta[k]) for k in range(num_barras if j != i else 0))
            elif i != 0 and j != 0:
                J[i, j] = Y_bus[i, j] * V[i] * V[j] * np.sin(theta[i] - theta[j])
                J[i, j + num_barras] = -Y_bus[i, j] * V[i] * V[j] * np.cos(theta[i] - theta[j])
    
    return J

def calcular_flujo_potencia(num_barras, datos_barras, tol=1e-8, max_iter=30):
    # Inicialización de datos
    V = np.ones(num_barras)
    theta = np.zeros(num_barras)
    
    # Definir la barra slack (barra 0)
    V[0] = 1.0  # Voltaje fijo en 1.0 pu
    theta[0] = 0.0  # Ángulo fijo en 0.0 grados
    
    iteracion = 0
    error = tol + 1
    
    # Obtener matriz de admitancia y potencias inyectadas
    Y_bus = calcular_matriz_admitancia(num_barras, datos_barras).astype(complex)
    P_iny, Q_iny = calcular_potencia_inyectada(num_barras, datos_barras)
    
    while error > tol and iteracion < max_iter:
        # Calcular Jacobiano
        J = calcular_jacobiano(num_barras, Y_bus, V, theta)
        
        # Calcular vector de potencias
        F = np.concatenate((np.dot(Y_bus, V) - P_iny, np.dot(Y_bus, V) - Q_iny))
        
        # Excluir la barra slack (barra 0) del vector de potencias
        F[0] = 0.0
        #depuracion
        print('Matriz J:')
        print(J)
        print('Vector F:')
        print(F)
        
        # Calcular incrementos delta(V) y delta(theta) usando el método de Newton-Raphson
        delta = np.linalg.solve(J, -F)
        
        delta_V = delta[:num_barras]
        delta_theta = delta[num_barras:]
        
        # Actualizar V y theta (excluir la barra slack)
        V[1:] += delta_V[1:]
        theta[1:] += delta_theta[1:]
        
        # Calcular error
        error = max(abs(F))
        
        # Incrementar contador de iteraciones
        iteracion += 1
    
    if error <= tol:
        print(f'Convergencia lograda en {iteracion} iteraciones.')
    else:
        print('El método no convergió.')
    
    # Devolver resultados finales (voltajes y ángulos)
    return V, np.degrees(theta)

In [33]:
R_km = 0.099
X_km = 0.156
Sb_VA = 100E6
Zb = (220E3**2)/Sb_VA
R_pu_km = R_km/Zb
X_pu_km = X_km/Zb 
P_pu = 150E6/Sb_VA
Q_pu = 150E6/Sb_VA
#Ejemplo de uso:
num_barras = 6
datos_barras = [
    #{'tipo': 'slack', 'P': 0.0, 'Q': 0.0, 'R': 0.0, 'X': 0.14},                          # Barra 1 (slack)
    {'tipo': 'PQ', 'P': 0, 'Q': 0, 'R': R_pu_km*10, 'X': X_pu_km*10},                    # Barra 2 (PQ)
    {'tipo': 'PQ', 'P': P_pu*0.20, 'Q': Q_pu*0.20, 'R': R_pu_km*15, 'X': X_pu_km*15},     # Barra 1A (PQ)
    {'tipo': 'PQ', 'P': P_pu*0.35, 'Q': Q_pu*0.35, 'R': R_pu_km*20, 'X': X_pu_km*20},     # Barra 2A (PQ)
    {'tipo': 'PQ', 'P': P_pu*0.15, 'Q': Q_pu*0.15, 'R': R_pu_km*15, 'X': X_pu_km*15},     # Barra 3A (PQ)
    {'tipo': 'PQ', 'P': P_pu*0.60, 'Q': Q_pu*0.60, 'R': R_pu_km*30, 'X': X_pu_km*30},     # Barra 2B (PQ)
    {'tipo': 'PQ', 'P': P_pu*0.10, 'Q': Q_pu*0.10, 'R': R_pu_km*10, 'X': X_pu_km*10},     # Barra 1B (PQ)
]

# Calcular el flujo de potencia
voltajes, angulos = calcular_flujo_potencia(num_barras, datos_barras)
print('Voltajes:', voltajes)
print('Ángulos (en grados):', angulos)

Matriz Y_bus antes de resolver:
[[ 140.3638281 -221.1793655j  -140.3638281 +221.1793655j
  -140.3638281 +221.1793655j  -140.3638281 +221.1793655j
  -140.3638281 +221.1793655j  -140.3638281 +221.1793655j ]
 [ -93.5758854 +147.45291033j   93.5758854 -147.45291033j
   -93.5758854 +147.45291033j  -93.5758854 +147.45291033j
   -93.5758854 +147.45291033j  -93.5758854 +147.45291033j]
 [ -70.18191405+110.58968275j  -70.18191405+110.58968275j
    70.18191405-110.58968275j  -70.18191405+110.58968275j
   -70.18191405+110.58968275j  -70.18191405+110.58968275j]
 [ -93.5758854 +147.45291033j  -93.5758854 +147.45291033j
   -93.5758854 +147.45291033j   93.5758854 -147.45291033j
   -93.5758854 +147.45291033j  -93.5758854 +147.45291033j]
 [ -46.7879427  +73.72645517j  -46.7879427  +73.72645517j
   -46.7879427  +73.72645517j  -46.7879427  +73.72645517j
    46.7879427  -73.72645517j  -46.7879427  +73.72645517j]
 [-140.3638281 +221.1793655j  -140.3638281 +221.1793655j
  -140.3638281 +221.1793655j  -140.363

C:\Users\Hp\AppData\Local\Temp\ipykernel_6716\2045804307.py:53: ComplexWarning: Casting complex values to real discards the imaginary part
  J[i, j] = Y_bus[i, j] * V[i] * V[j] * np.sin(theta[i] - theta[j])
C:\Users\Hp\AppData\Local\Temp\ipykernel_6716\2045804307.py:54: ComplexWarning: Casting complex values to real discards the imaginary part
  J[i, j + num_barras] = -Y_bus[i, j] * V[i] * V[j] * np.cos(theta[i] - theta[j])


LinAlgError: Singular matrix